# Tugas 1 - Lab Basis Data - Data Scraping

Website: https://www.brighton.co.id/cari-properti/?OrderBy=5&sortRadio=on&Latitude=&Longitude=

<p> Nama: Muhammad Zaky
<br> NIM: 18220071

## Import Important Libraries

In [1]:
import json
from bs4 import BeautifulSoup
import requests
import csv
import time
from tqdm import tqdm
import os
import pandas as pd

## Permasalahan

<i>Website</i> yang akan Saya <i>scrape</i> ini membatasi <i>guest</i> untuk hanya dapat melakukan akses terhadap <i>website</i> hingga laman ke-8. Sehingga, ketika <i>website</i> sudah mencapai laman ke-9, maka <i>guest</i> tersebut harus melakukan <i>login</i> ke dalam <i>website</i>. Maka dari itu, kita perlu menambahkan beberapa informasi, seperti <i>cookies</i> dan <i>header</i> agar dapat mengakses <i>website</i> secara keseluruhan.

In [2]:
cookies = {
    '_gcl_au': '1.1.1740225377.1655203706',
    '_ga': 'GA1.3.585457613.1655203706',
    '_gid': 'GA1.3.1404984123.1655203706',
    '_fbp': 'fb.2.1655203706505.1927761622',
    'ViewProductID3298051': '3298051',
    'ViewProductID3298054': '3298054',
    'PHPSESSID': '4f3f52a7557ed7ee465c7d69eeafc49b',
    'SESSIONCOOKIE_2': 'MTY1NTI2ODE5NzM3MjYwUHVibGljTWVtYmVyRGF0YQ%3D%3D',
    'PHPSESSID_2': 'CPupCEQRSXPlOQxgPEkYsiYgstgbfnV5oPF3lagWdcW5hZbu1BcYOcqB6y2INuYTCdzCZuHdkOf9kifqVZU1tNSe84xdNgM8EfWUnOSK7pBQUGObd%2FVSJ12M2Hh8tMDZlBpDW1uhf63%2B%2FSDXOnre%2BrwSt5uCRSQBoYWCT7kMqIbcqM0hCGdIxNtBIA4%2BV3%2F6y%2FL1vAjaZFya45OMFg%2Bf%2Fp3KHjOPcDyv3%2BFfPu0LSeKsrsEvG1b1H3ijN38rgp%2FNnQZcbP66NGsxDepTEco9H8rKnriFRG6%2FBbJ04NLWDAIzOOcjDd6z%2FwkI8n6OSQsSnUtPoTo%2Fko4py%2B4YVsxfjQ%3D%3D',
}

headers = {
    'authority': 'www.brighton.co.id',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    # Requests sorts cookies= alphabetically
    # 'cookie': '_gcl_au=1.1.1740225377.1655203706; _ga=GA1.3.585457613.1655203706; _gid=GA1.3.1404984123.1655203706; _fbp=fb.2.1655203706505.1927761622; ViewProductID3298051=3298051; ViewProductID3298054=3298054; PHPSESSID=4f3f52a7557ed7ee465c7d69eeafc49b; SESSIONCOOKIE_2=MTY1NTI2ODE5NzM3MjYwUHVibGljTWVtYmVyRGF0YQ%3D%3D; PHPSESSID_2=CPupCEQRSXPlOQxgPEkYsiYgstgbfnV5oPF3lagWdcW5hZbu1BcYOcqB6y2INuYTCdzCZuHdkOf9kifqVZU1tNSe84xdNgM8EfWUnOSK7pBQUGObd%2FVSJ12M2Hh8tMDZlBpDW1uhf63%2B%2FSDXOnre%2BrwSt5uCRSQBoYWCT7kMqIbcqM0hCGdIxNtBIA4%2BV3%2F6y%2FL1vAjaZFya45OMFg%2Bf%2Fp3KHjOPcDyv3%2BFfPu0LSeKsrsEvG1b1H3ijN38rgp%2FNnQZcbP66NGsxDepTEco9H8rKnriFRG6%2FBbJ04NLWDAIzOOcjDd6z%2FwkI8n6OSQsSnUtPoTo%2Fko4py%2B4YVsxfjQ%3D%3D',
    'if-modified-since': 'Tue, 14 Jun 2022 13:21:19 GMT',
    'if-none-match': '"c2881c4df9f77a40b000206b3ecfc35d"',
    'referer': 'https://www.brighton.co.id/cari-properti/?OrderBy=5&sortRadio=on&Latitude=&Longitude=&page=1',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36',
}

params = {
    'OrderBy': '5',
    'sortRadio': 'on',
    'Latitude': '',
    'Longitude': '',
    'page': '2',
}

## Pendefinisian Fungsi

In [3]:
'''Melakukan convert harga dari string menjadi integer.'''
def harga_integer(harga):
    split_harga = harga.split(',') #Menghilangkan tanda koma sebagai tanda pemisah dari harga.
    harga_tanpa_koma = ''
    for i in range (len(split_harga)):
        harga_tanpa_koma += split_harga[i] #Menambahkan keseluruhan string, sehingga harga_tanpa_koma akan berisi harga yang sudah tidak ada pemisah berupa tanda koma.
    harga_int = int(harga_tanpa_koma.split(' ')[1]) #Mengambil hanya angkanya saja dari harga_tanpa_koma tanpa ada satuan Rp dan melakukan converting menjadi integer.
    return harga_int

In [4]:
'''Mengembalikan satuan berupa Miliar atau Juta dari harga'''
def unitharga(harga):
    if(len(str(harga)) >= 10) & (len(str(harga)) <= 12):
        text = 'Miliar'
    elif(len(str(harga)) >= 7) & (len(str(harga)) <= 9):
        text = 'Juta'
    return text

## Permasalahan Ketika Scraping

Disini Saya mengambil beberapa informasi dari website tersebut beserta permasalahannya, yaitu:
1. <b>Label</b>: Ada kemungkinan properti tersebut tidak memiliki label (Featured, Premier).
2. <b>Link</b>.
3. <b>Nama Properti</b>: Setiap nama properti pada website ini diawali dengan kata Jual.
4. <b>Description</b>.
5. <b>Jumlah Kamar Tidur</b>: Ada kemungkinan jumlah kamar tidur ini berupa string penjumlahan (contoh: 3+1).
6. <b>Jumlah Kamar Mandi</b>: Ada kemungkinan jumlah kamar mandi ini berupa string penjumlahan (contoh: 3+1).
7. <b>Luas Tanah</b>: Ada kemungkinan luas tanah tidak ada.
8. <b>Luas Bangunan</b>: Ada kemungkinan luas bangunan tidak ada.
9. <b>Nama Agen</b>.
10. <b>Unit Agen</b>.

In [5]:
'''Fungsi utama yang akan melakukan web scraping.'''
def scraping_pages(object, firstPage, lastPage):
    for page in range(firstPage, lastPage+1):
        pages = requests.get("https://www.brighton.co.id/cari-properti/?OrderBy=5&sortRadio=on&Latitude=&Longitude=&page=" + str(page), params=params, cookies=cookies, headers=headers)
        soup = BeautifulSoup(pages.text, 'html.parser')
        properti = soup.findAll('div', 'col-12 col-md-6 col-lg-4 py-2 px-0 py-sm-2 px-md-2')
        for item in tqdm(properti):
            try:
                label = item.p.text.strip().upper()
            except AttributeError:
                label = ''
            link = item.find('a').get('href')
            nama = item.h2.text
            nama = nama[5:].upper() #Menghilangkan bagian kata 'Jual '
            desc = item.h3.text.strip().upper()
            try:
                kamar_tidur = item.find('span', {'title': 'Kamar Tidur'}).text
                if(kamar_tidur.__contains__('+')):
                    kamar_tidur = int(kamar_tidur.split('+')[0])+int(kamar_tidur.split('+')[1]) #Apabila terdapat '+' di kamar_tidur, maka akan dilakukan splitting, sehingga kita akan mendaptkan angkanya saja dan bisa dijumlah
                else:
                    kamar_tidur = int(kamar_tidur) #Apabila tidak terdapat '+', maka akan langsung di convert ke integer.
            except AttributeError:
                kamar_tidur = ''
            except ValueError:
                kamar_tidur = ''

            try:
                kamar_mandi = item.find('span', {'title': 'Kamar Mandi'}).text
                if(kamar_mandi.__contains__('+')):
                    kamar_mandi = int(kamar_mandi.split('+')[0])+int(kamar_mandi.split('+')[1]) #Apabila terdapat '+' di kamar_mandi, maka akan dilakukan splitting, sehingga kita akan mendaptkan angkanya saja dan bisa dijumlah
                else:
                    kamar_mandi = int(kamar_mandi) #Apabila tidak terdapat '+', maka akan langsung di convert ke integer.
            except AttributeError:
                kamar_mandi = ''

            try:
                luas_tanah = float(item.find('span', {'title':'Luas Tanah'}).text[:-2]) #Menghilangkan satuan m2 pada string dan melakukan converting string menjadi float.
            except AttributeError:
                luas_tanah = ''

            try:    
                luas_bangunan = float(item.find('span', {'title': 'Luas Bangunan'}).text[:-2]) #Menghilangkan satuan m2 pada string dan melakukan converting string menjadi float.
            except AttributeError:
                luas_bangunan = ''

            harga = item.find('p', 'harga-properti').text
            harga_int = harga_integer(harga)
            unit = unitharga(harga_int).upper() #Mendaptkan unit harga dari harga (Juta atau Miliar).
            if(unit == 'MILIAR'):
                harga_int = harga_int/10**9
            elif (unit == 'JUTA'):
                harga_int = harga_int/10**6

            nama_agen = item.h4.text.upper()
            unit_agen = item.find('p', 'unit-agen').text.upper()


            value = {
                "Nama": nama, 
                "Label": label,
                "Deskripsi": desc,
                "Luas Tanah(m2)": luas_tanah, 
                "Luas Bangunan(m2)": luas_bangunan,
                "Jumlah Kamar Tidur": kamar_tidur,
                "Jumlah Kamar Mandi": kamar_mandi, 
                "Harga": harga_int, 
                "Unit Harga": unit, 
                "Nama Agen": nama_agen, 
                "Unit Agen": unit_agen, 
                "Link": link
            }
            object.append(value)

## Tahap Pengambilan Data

In [ ]:
values = []
scraping_pages(values, 1, 100)
print("Berhasil melakukan scraping data sebanyak "+len(values))

100%|██████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 1002.68it/s]


In [7]:
len(values) #Memeriksa jumlah data yang berhasil diambil.

90

In [9]:
databersih = [i for n, i in enumerate(values) if i not in values[n + 1:]]

In [10]:
len(databersih) #Memeriksa jumlah data yang benar-benar diambil.

90

In [12]:
filename = input("Nama File: ")
filepath = 'G:/My Drive/Tugas Lab Basis Data/Brighton Web Scraping/data'
dump = json.dumps(databersih, indent=4)
filepath = os.path.join(filepath, filename + ".json")
with open(filepath, "w") as file:
    file.write(dump)
    file.close()

Nama File:  brightonData
